In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import random
import ipyplot
import math

from utils import *
from bbox import *

load_dotenv()
Migrator().run()

In [ ]:
query = "wedding*moose"
models = Model.find(Model.filename % query).all()
models = [model for model in models if model.faces]
models = [model for model in models if len(model.faces) == 5]

print(len(models))

sample_model = random.sample(models, 1)[0]
model_crop_and_rotate_and_resize_clips = crop_and_rotate_and_resize_clip(sample_model)
ipyplot.plot_images(model_crop_and_rotate_and_resize_clips)
sample_model

In [ ]:
def new_rotated_bbox(model):
    image = Image.open(model.filename)
    w, h = image.size
    cx, cy = w // 2, h // 2

    rotated_bboxes = []

    for i, face in enumerate(model.faces):
        bboxes = np.array(
            [(face.bbox.x1, face.bbox.y1, face.bbox.x2, face.bbox.y2)], dtype="float"
        )
        corners = get_corners(bboxes)
        corners = np.hstack((corners, bboxes[:, 4:]))
        img_rotated = image.rotate(face.rotation, expand=1)

        corners[:, :8] = rotate_box(corners[:, :8], face.rotation, cx, cy, h, w)
        new_bbox = get_enclosing_box(corners)
        scale_factor_x = img_rotated.size[0] / w
        scale_factor_y = img_rotated.size[1] / h

        img_rotated = img_rotated.resize((w, h))

        new_bbox[:, :4] /= [
            scale_factor_x,
            scale_factor_y,
            scale_factor_x,
            scale_factor_y,
        ]
        bboxes = new_bbox
        bboxes = clip_box(bboxes, [0, 0, w, h], 0.25).squeeze()

        model.faces[i].bbox.x1 = int(round(bboxes[0], 0))
        model.faces[i].bbox.y1 = int(round(bboxes[1], 0))
        model.faces[i].bbox.x2 = int(round(bboxes[2], 0))
        model.faces[i].bbox.y2 = int(round(bboxes[3], 0))
        model.save()

    return model

In [ ]:
modified_faces = crop_and_rotate_and_resize_clip(new_rotated_bbox(sample_model))

ipyplot.plot_images(modified_faces)